<a href="https://colab.research.google.com/github/mkbahk/graphcore-ipu-demo/blob/main/mnist-dnn-keras-functional-cpu-input-and-output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import time, os

In [123]:
start = time.time() # 시작 시간 저장
strategy = tf.distribute.get_strategy()

In [124]:
# The input data and labels.
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, x_test) = (x_train / 255.0, x_test / 255.0)

# Add a channels dimension.
x_train = x_train[..., tf.newaxis]
print(x_train.shape)
x_test = x_test[..., tf.newaxis]
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [125]:
def create_train_dataset():
    print("==============================Processing Training DataSet==============================\n\n")
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(32, drop_remainder=True)
    print(train_ds)
    train_ds = train_ds.map(lambda data, label: (tf.cast(data, tf.float32), tf.cast(label, tf.float32)))
    print(train_ds)
    return train_ds.repeat()
#end of def

def create_test_dataset():
    print("==============================Processing Test  DataSet==============================\n\n")
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(10000).batch(32, drop_remainder=True)
    print(test_ds)
    test_ds = test_ds.map(lambda data, label: (tf.cast(data, tf.float32), tf.cast(label, tf.float32)))
    print(test_ds)
    return test_ds.repeat()
#end of def

In [126]:
def create_model():
    
    inputs = tf.keras.Input(shape = (28, 28))
    
    flatten_layer = keras.layers.Lambda(lambda ipt: K.reshape(ipt, (-1, 28 * 28)))
    flattened_input = flatten_layer(inputs)

    x = keras.layers.Flatten()(flattened_input)   
    x = keras.layers.Dense(128, activation='relu')(x) 
    x = keras.layers.Dense(256, activation='relu')(x)
    x = keras.layers.Dense(128, activation='relu')(x)

    outputs = keras.layers.Dense(10, activation='softmax')(x)
    
    # Defined the model.
    model = tf.keras.Model(inputs, outputs, name="dnn")
    
    model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                   optimizer = tf.keras.optimizers.Adam(), 
#                  steps_per_execution = 50, 
                   metrics=['sparse_categorical_accuracy'])
    return model
#end of def

In [127]:
def main():
      # Get the training dataset.
      print("==============================Getting Training DataSet=================================\n\n")
      ds1 = create_train_dataset()
      print("==============================Getting Test DataSet====================================\n\n")
      ds2 = create_test_dataset()

      with strategy.scope():   
           # Create an instance of the model.
           print("==============================Building Model====================================\n\n")
           model = create_model()

           model.summary()

           print("==============================Building Compile====================================\n\n")      

        
           print("==============================Model Training ====================================\n\n")
           model.fit(ds1, steps_per_epoch=2000, epochs=50)

           print("\n\n==============================Checking the result==============================\n\n")
           (loss, accuracy) = model.evaluate(ds2, steps=1000)
        
           print("Validation loss: {}".format(loss))
           print("Validation accuracy: {}%".format(100.0 * accuracy))
           print("\n\n==============================Job Done====================================")
      #end of with
#end of def

In [128]:
if __name__ == '__main__':
    main()
#end of if

==============================Getting Training DataSet=================================


==============================Processing Training DataSet==============================


<BatchDataset shapes: ((32, 28, 28, 1), (32,)), types: (tf.float64, tf.uint8)>
<MapDataset shapes: ((32, 28, 28, 1), (32,)), types: (tf.float32, tf.float32)>
==============================Getting Test DataSet====================================


==============================Processing Test  DataSet==============================


<BatchDataset shapes: ((32, 28, 28, 1), (32,)), types: (tf.float64, tf.uint8)>
<MapDataset shapes: ((32, 28, 28, 1), (32,)), types: (tf.float32, tf.float32)>
==============================Building Model====================================


Model: "dnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 28, 28)]          0         
__________________________________

In [129]:
print("Total Execution Time :", time.time() - start,"(Sec)")

실행시간 : 318.5923638343811 (초)
